In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [106]:
class BuySell:
    def __init__(self, capital=None, max_share=None):
        self.original_capital = capital
        self.capital = capital
        self.max_share = max_share
        self.share = 0
        self.is_holding = False
        self.buy_at = 0
        # Array for dollar gain per trade 
        self.dollar_gain = []
        # Array for percentage gain per trade
        self.pct_gain = []
        # Array for Buy / Sell price
        self.stat = []
    
    def buy(self, price):
        # Buy Action
        self.stat.append([price, np.nan])
        self.buy_at = price
        self.is_holding = True
        # Buy Power with capital
        if self.capital is not None:
            self.share = self.capital // self.buy_at
            if self.max_share is not None and self.share > self.max_share:
                self.share = self.max_share
            self.capital -= np.round(self.buy_at * self.share, 2)
    
    def sell(self, price):
        # Sell Action
        self.calculate_gain(price)
        self.stat.append([np.nan, price])
        if self.capital is not None:
            self.capital += np.round(price * self.share)
            self.share = 0
        self.buy_at = 0
        self.is_holding = False
    
    def no_action(self):
        self.stat.append([np.nan, np.nan])
    
    def calculate_gain(self, price):
        dollar_gain = price - self.buy_at
        pct_gain = price / self.buy_at - 1
        self.dollar_gain.append(dollar_gain)
        self.pct_gain.append(pct_gain)
    
    def show_results(self):
        num_trades = len(self.dollar_gain)
        total_dollar_gain = sum(self.dollar_gain)
        total_pct_gain = sum(self.pct_gain)
        print('${:.2f} gained per share after {} trades'.format(total_dollar_gain, num_trades))
        print('and have a total gain percentage of {:.2f}%'.format(total_pct_gain * 100))
        results = [num_trades, total_dollar_gain, total_pct_gain]
        if self.capital is not None:
            capital_gain = self.capital / self.original_capital - 1
            print('Test Ending Capital: ${:.2f}'.format(self.capital))
            print('With {:.2f}% Capital Gain'.format(capital_gain * 100))
            results.append(capital_gain)
        self.results = self.result_Series(results)
        
    def result_Series(self, array):
        index = ['Num-Trades', 'Total-Dollar-Gain', 'Total-Percentage-Gain', 'Percentage-Capital-Gain']
        if self.capital is not None:
            return pd.Series(array, index=index)
        return pd.Series(array, index=index[:-1])
        
        
        

In [119]:
d = DataLoader('MSFT', '2018-01-01', '2020-12-31')
prices = d.data[['Close', 'Low', 'High']]

In [120]:
# MA Crossing
a, b = 7, 14
fast_ma = prices['Close'].rolling(a).mean()
slow_ma = prices['Close'].rolling(b).mean()
above = fast_ma > slow_ma
data = pd.concat([fast_ma, slow_ma, above], axis=1)
data.columns = ['fast_ma', 'slow_ma', 'above']
data = pd.concat([prices, data], axis=1)
data = data.dropna()
data

,Close,Low,High,fast_ma,slow_ma,above
Date,,,,,,
2018-01-22,91.610001,89.739998,91.620003,89.697142,88.557142,True
2018-01-23,91.900002,91.540001,92.300003,90.242857,88.982143,True
2018-01-24,91.820000,91.580002,93.430000,90.560000,89.372857,True
2018-01-25,92.330002,91.930000,93.239998,91.128572,89.745714,True
2018-01-26,94.059998,92.580002,94.059998,91.688572,90.165000,True
...,...,...,...,...,...,...
2020-12-24,222.750000,221.199997,223.610001,221.084285,217.271428,True
2020-12-28,224.960007,223.020004,226.029999,221.895715,218.033571,True
2020-12-29,224.149994,223.580002,227.179993,222.571428,218.615000,True


In [121]:
length = len(data)
bs = BuySell(capital=1000)

for i in range(length):
    is_above = data['above'].iloc[i]
    low = data['Low'].iloc[i]
    high = data['High'].iloc[i]
    close = data['Close'].iloc[i]
    if i + 1 == length:
        if bs.is_holding:
            bs.sell(high)
    else:
        if not bs.is_holding:
            if is_above:
                bs.buy(low)
            else:
                bs.no_action()
        else:
            if not is_above:
                bs.sell(close)
            else:
                bs.no_action()


In [122]:
stats = np.array(bs.stat)
data['Buy'] = stats[:, 0]
data['Sell'] = stats[:, 1]
data

,Close,Low,High,fast_ma,slow_ma,above,Buy,Sell
Date,,,,,,,,
2018-01-22,91.610001,89.739998,91.620003,89.697142,88.557142,True,89.739998,NaN
2018-01-23,91.900002,91.540001,92.300003,90.242857,88.982143,True,NaN,NaN
2018-01-24,91.820000,91.580002,93.430000,90.560000,89.372857,True,NaN,NaN
2018-01-25,92.330002,91.930000,93.239998,91.128572,89.745714,True,NaN,NaN
2018-01-26,94.059998,92.580002,94.059998,91.688572,90.165000,True,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-12-24,222.750000,221.199997,223.610001,221.084285,217.271428,True,NaN,NaN
2020-12-28,224.960007,223.020004,226.029999,221.895715,218.033571,True,NaN,NaN
2020-12-29,224.149994,223.580002,227.179993,222.571428,218.615000,True,NaN,NaN


In [123]:
bs.show_results()

$121.68 gained per share after 31 trades
and have a total gain percentage of 87.94%
Test Ending Capital: $2181.77
With 118.18% Capital Gain


In [124]:
bs.results

Num-Trades                  31.000000
Total-Dollar-Gain          121.679985
Total-Percentage-Gain        0.879411
Percentage-Capital-Gain      1.181770
dtype: float64